In [ ]:
from math import sin, cos, sqrt, atan2, radians
import matplotlib.pyplot as plt

In [ ]:
cord_drone = (50.797687, 8.762331)
cord_dest = (50.797212, 8.763172)
cord_test = (50.797685, 8.762335)

print(vertDist(cord_drone, cord_dest))
print(horiDist(cord_drone, cord_dest))
getDist(cord_dest, cord_drone)

In [ ]:
from geopy.distance import distance

def getDist(cord_drone, cord_dest):
    """
    Gibt den Abstand zwischen der Drohne und dem Zielpunkt in metern zurück
    https://geopy.readthedocs.io/en/stable/#
    :param cord_drone: Koordinate der Drohne im Format (log, lat)
    :param cord_dest: Koordinate des Zielpunkts im Format (log, lat)
    :return Abstand in Metern
    """
    return distance(cord_dest, cord_drone).m

def vertDist(cord_root, cord_dest):
    temp = (cord_dest[0], cord_root[1])
    if cord_root[0] < cord_dest[0]:
        return distance(cord_root, temp).m
    else:
        return -distance(cord_drone, temp).m
    

def horiDist(cord_root, cord_dest):
    temp = (cord_root[0], cord_dest[1])
    if cord_root[1] < cord_dest[1]:
        return distance(cord_root, temp).m
    else:
        return -distance(cord_root, temp).m

In [ ]:
import numpy as np


class path_map:
    
    def __init__(self, cord_drone, cord_dest, radius=5, pixel_size=1):
        self.root = cord_drone
        self.cord_drone = cord_drone
        self.pixel_size = pixel_size

        width = horiDist(cord_drone, cord_dest)/pixel_size
        height = vertDist(cord_drone, cord_dest)/pixel_size
        
        width = round(width + (2*radius*width)/(abs(width)*pixel_size))
        height = round(height + (2*radius*height)/(abs(height)*pixel_size))
        
        # print(width, height)

        self.drone_pos = [round(width*(radius/pixel_size)/abs(width)), round(-height*(radius/pixel_size)/abs(height))]
        dest_pos = [round(-width*(radius/pixel_size)/abs(width)), round(height*(radius/pixel_size)/abs(height))]

        self.width = abs(width)
        self.height = abs(height)
        # print(self.drone_pos, dest_pos)

        self.path_map = [[0 if ((i != 0) and (i != self.width - 1) and (j != 0) and (j != self.height-1)) else 0.5 for i in range(self.width)] for j in range(self.height)]
        
        # range normalisieren ([0,1]), damit der Plot die Farben richtig anfängt
        self.path_map[1][1] = 1
        
        # Farbe des Ziels und der Drohne setzen
        self.path_map[self.drone_pos[0]][self.drone_pos[1]] = 0.75
        self.path_map[dest_pos[0]][dest_pos[1]] = 0.25
        # print(len(self.path_map), len(self.path_map[0]))
    
    def add_cord(self, cord, lable):
        """
        Füngt eine Koordinate (lat, lon) der map hinzu, wenn diese innerhalb des Rahmens liegt
        :param cord: Koordinate des Punktes der hinzugefügt werden soll
        :param lable: Label der Koordinate (0 = Hinderniss, 0.75 = Drohne, 0.25 = Ziel)
        
        Wahrscheinlich nur nützlich um mehrere Ziele hinzuzufügen, bzw für Checkpoints o.Ä
        """
        vDist = round(vertDist(self.root, cord)/self.pixel_size)
        hDist = round(horiDist(self.root, cord)/self.pixel_size)
        
        if (0 < (self.drone_pos[0] - vDist) < self.height) and (0 < (self.drone_pos[1] + hDist) < self.width):
            self.path_map[self.drone_pos[0] - vDist][self.drone_pos[1] + hDist] = lable
    
    
    def change_drone_pos(self, cord_new):
        """
        Ändert die Position der Drohne auf der Karte
        :param cord_new: Neue Koordinate der Drohne (lat, log)
        """
        vDist = round(vertDist(self.cord_drone, cord_new)/self.pixel_size)
        hDist = round(horiDist(self.cord_drone, cord_new)/self.pixel_size)
        if (0 < (self.drone_pos[0] - vDist) < self.height) and (0 < (self.drone_pos[1] + hDist) < self.width):
            self.cord_drone = cord_new
            
            # remove current drone point
            self.path_map[self.drone_pos[0]][self.drone_pos[1]] = 0
            
            # add new drone point
            self.path_map[self.drone_pos[0] - vDist][self.drone_pos[1] + hDist] = 0.75
            
            # update pos_drone
            self.drone_pos = [self.drone_pos[0] - vDist, self.drone_pos[1] + hDist]
            
    def add_vec(self, vec, lable):
        """
        Fügt ein vector hinzu. Der Vektor enthält die x/y Abstände des Punkten im Verhältnis zur Drohne
        :param vec: Vektor mit (x,y) Abstände in Metern
        :param lable: Label der Koordinate (0 = Hinderniss, 0.75 = Drohne, 0.25 = Ziel)
        """
        x = round(vec[0]/self.pixel_size)
        y = round(vec[1]/self.pixel_size)
        if (0 < (self.drone_pos[0] - y) < self.height) and (0 < (self.drone_pos[1] + x) < self.width):
            self.path_map[self.drone_pos[0] - y][self.drone_pos[1] + x] = lable
        
    

dmap = path_map(cord_drone, cord_dest);

dmap.add_cord((50.797394, 8.762707), 0.5)
dmap.add_cord((50.797338, 8.762817), 0.5)
dmap.add_cord((50.797238, 8.762568), 0.5)
dmap.add_cord((50.797674, 8.762606), 0.5)
dmap.add_vec((10, 0), 0.5) # -> 5, 15
dmap.change_drone_pos((50.797459, 8.762391))
dmap.add_vec((10, -6), 0.5) # -> ~ 18, 35
# https://maps.openrouteservice.org/directions?n1=50.797671&n2=8.762603&n3=18&a=50.797687,8.762331,50.797238,8.762568,50.797674,8.762606,50.797394,8.762707,50.797338,8.762817,50.797238,8.762568,50.797212,8.763172&b=2&c=0&k1=en-US&k2=km

fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
plt.imshow(dmap.path_map, cmap='twilight');